## 대회 개요

- 주택 가격을 예측하는 대회
- 목적 변수: 주택 가격
- 주택 1460채의 정보가 학습 데이터로 주어지고, 이 데이터를 바탕으로 모델을 작성한 후 주택 1459채의 가격을 예측함
- 예측 정확도 평가는 평균 제곱 오차(Root Mean Squared Error, RMSE)를 사용하며, 정답 데이터와 차이가 작을수록 상위를 차지함
- 평가 항목 참고: Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally
- RMSE를 사용할 때는 예측 값과 실제 값 차이가 너무 큰 데이터가 포함되어 있으면 다른 예측 값의 정확도가 아무리 좋아도 전체 예측 정확도가 크게 영향을 받음
- 반면 실제 값, 예측 값 각각에 로그를 취하면 실제 값, 예측 값 간 차이가 작아져 특정 데이터가 전체 평가에 영향을 미치는 정도를 줄일 수 있음

### 분류 classification vs. 회귀 regression

1. classification: 클래스 두 개 혹은 여러 개로 분류하는 것
2. regression: 연속된 값을 예측하는 것

In [1]:
# 필요한 라이브러리 불러오기

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.style.use("ggplot")

In [3]:
import pandas as pd
import numpy as np

In [4]:
# 랜덤 시드 설정하기

import random
np.random.seed(1234)
random.seed(1234)

In [5]:
# csv data 불러오기

train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
submission = pd.read_csv("./data/sample_submission.csv")

In [6]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 벤치마크용 베이스라인 작성

- 모델을 작성하려면 주어진 설명 변수를 확인하거나 데이터의 분포, 결측치를 탐생하는 등 전처리 과정을 진행해야 함
- 먼저 최소한의 카테고리 변수를 라벨 인코딩 처리한 후 LightGBM에 넣었을 때 정확도가 얼마인지 알아보자
- 이후 처리가 정확도 향상에 얼마나 효과가 있었는지를 검증하기 위해서

### LightBGM 으로 예측

- LightBGM은 각 변수의 중요도를 나타낼 수 있기 때문에 향후 처리의 우선순위 등을 생각하는 지표로 활용될 수 있음
- 중요도가 높은 변수들을 전처리하는 것 이외에 중요한 변수로 보이지만 중요도는 낮게 나오는 변수들에 대한 처리 과정도 필요함

In [7]:
# 학습 데이터의 변수 유형 확인하기

train_df.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [8]:
# 각 항목에 얼마나 많은 수의 값이 존재하는지 알아보기

# MSZoning: object type: "Identifies the general zoning classification of the sale": 상업용, 주거용 등 판매처 용도의 분류

train_df["MSZoning"].value_counts()

MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: count, dtype: int64

- 이와 같이 데이터 유형이 object인 것은 문자열 데이터라고 하는 카테고리 변수임
- LightBGM에서 읽을 수 있는 것은 int형, float형, bool형이기 때문에 문자열을 카테고리 변수로 변환해야 함